In [49]:
import fourier_hardware_py
config = fourier_hardware_py.HardwareConfigHelper("/usr/local/config/fourier_hardware/gr2t2v2")
# 配置文件中有灵巧手相关字段 
config.getHandNames()


['left_hand', 'right_hand']

In [50]:
left_hand_type = config.getHandType("left_hand")
# 灵巧手类型，有FDH-6L,FDH-6R,FDH-12L,FDH-12R
left_hand_type

'FDH-6L'

In [51]:
right_hand_type = config.getHandType("right_hand")
right_hand_type

'FDH-6R'

In [52]:
# 根据灵巧手的名字获得灵巧手的ip
left_hand_ip = config.getHandIP("left_hand")
left_hand_ip

'192.168.137.19'

In [53]:
right_hand_ip = config.getHandIP("right_hand")
right_hand_ip

'192.168.137.39'

In [54]:
# 构造灵巧手的控制对象
dexhand = fourier_hardware_py.DexHand()

[2025-05-30 05:44:19.327] [I:fhand.cpp:15] fhand version: 0.2.0


In [55]:
# 灵巧手init()，注意可以根据返回值判断是否init成功
ret = dexhand.init()
ret

[2025-05-30 05:44:20.347] [W:fhand.cpp:164] broadcase receive timeout


<DexRet.FAIL: -1>

[2025-05-30 05:44:20.347] [E:fhand.cpp:228] Connot find server


In [56]:
if ret == fourier_hardware_py.DexRet.FAIL:
    print("dexhand init failed")
elif ret == fourier_hardware_py.DexRet.SUCCESS:
    print("dexhand init success")

dexhand init failed


In [57]:
# 获取灵巧手init时检测到的ip，可以配置文件中的ip比较，确定ip是否正确
ip_list = dexhand.get_ip_list()
ip_list

[]

In [58]:
# 获取灵巧手位置，返回一个数组，空代表获取失败，最大等待时间100ms 
# 50 hz 频率get pos，1000次中平均有10次超时，超时时间平均为0.022s，没有发现大规模超时，平均单手get时间约为0.01s，建议一手一线程，get和set不要再一个线程。
# set线程很快，不会造成线程堵塞，但是会受到get函数的影响。
import threading
import time
import math

left_hand_ip = "192.168.137.19"
right_hand_ip = "192.168.137.39"

def hand_loop(name, ip):
    count = 0
    start_time = time.perf_counter()
    phase = 0.0

    while True:
        loop_start = time.perf_counter()

        # 控制值可变化（模拟控制）
        phase += 0.02
        command = (math.sin(phase) + 1) / 2
        command_array = [command]*5 + [-1]

        pos = dexhand.get_pos(ip)
        # print(f"[{name}] get_pos耗时: {(time.perf_counter() - loop_start)*1000:.2f} ms")
        dexhand.set_pos(ip, command_array)

        count += 1
        now = time.perf_counter()

        if now - start_time >= 1.0:
            freq = count / (now - start_time)
            # print(f"[{name}] 频率: {freq:.2f} Hz")
            start_time = now
            count = 0

        # 精准睡眠（尝试压到100Hz）
        duration = time.perf_counter() - loop_start
        if duration < 0.01:
            time.sleep(0.01 - duration)

# 启动两个线程
threading.Thread(target=hand_loop, args=("left", left_hand_ip), daemon=True).start()
threading.Thread(target=hand_loop, args=("right", right_hand_ip), daemon=True).start()


In [59]:
# 设置灵巧手位置，返回只代表消息是否发送成功，可能由于丢包导致设置失败，可以通过get_pos函数判断网络是否畅通 ，-1 代表不控制该自由度
# 六自由度灵巧手控制范围为0-1
# 12自由度范围为
# 食指1-3：0-1750，0-1780，0-576
# 中指4-5：0-1750，0-1780
# 无名指6-7：0-1750， 0-1780
# 小指8-9：0-1750， 0-1780
# 拇指10-12：0-1700，0-1700，0-1700
ret = dexhand.set_pos(left_hand_ip,[0.9,-1,-1,-1,0.5,-1])

In [60]:
dexhand.clear_errorcode(left_hand_ip)
dexhand.get_errorcode(left_hand_ip)

[2025-05-30 05:44:20.385] [W:fhand.cpp:272] clear_errorcode 192.168.137.19 IP not found.
[2025-05-30 05:44:20.385] [W:fhand.cpp:258] get_errorcode 192.168.137.19 IP not found.


[]

In [61]:
# pvc 单位为rad,rad/s,mA
pvc = dexhand.get_pvc(left_hand_ip)
print(pvc)

[]


In [62]:
dexhand.set_pvc(left_hand_ip,pvc)

<DexRet.FAIL: -1>

In [63]:
# 上电自动enable
# disable之后需要enable
dexhand.enable()

<DexRet.SUCCESS: 0>

In [64]:
# 12 自由度可用,去使能
dexhand.disable()

<DexRet.SUCCESS: 0>